In [155]:
import warnings
warnings.filterwarnings('ignore')

import librosa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Flatten, Reshape, TimeDistributed, GlobalAveragePooling2D,GlobalAveragePooling1D, Bidirectional
    
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

In [30]:
Audio?

In [31]:
Audio(clips + '/' + 'common_voice_en_38491520.mp3')

In [14]:
df = pd.read_csv('common_voice/en/other.tsv', sep = '\t')

#number of samples per target
value_counts = df['client_id'].value_counts()

# targets > 1000 datapoints
targets = np.array(value_counts[value_counts > 1000].index)

usable_samples = df.loc[df['client_id'].isin(targets)]

In [19]:
clips = 'common_voice/en/clips'

In [20]:
# Normalizing the matrices
def matrix_standard_scaler(matrix):
    mean = matrix.mean()
    std = matrix.std()
    if std == 0:
        std = 1e-8
    norm_matrix = (matrix - mean) / std
    return norm_matrix

In [25]:
def get_mfcc(audio_path:str, target_duration = 7):
    # Load audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Clip or zero-pad audio to match target duration
    if len(audio) > sr * target_duration:
        # Clip audio to target duration
        audio = audio[:sr * target_duration]
    elif len(audio) < sr * target_duration:
        # Zero-pad audio to target duration
        pad_width = sr * target_duration - len(audio)
        audio = np.pad(audio, (0, pad_width), mode='constant')

        # Define the replacement value (in dB)
        replacement_value = -80

        # Find indices of non-zero values
        non_zero_indices = audio != 0

        # Replace zeros with the chosen value
        audio[~non_zero_indices] = replacement_value

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, hop_length = 2000)
    norm_mfccs = matrix_standard_scaler(mfccs)

    return norm_mfccs
    

In [28]:
audio = clips + "/" + "common_voice_en_38491520.mp3"

(20, 113)

In [33]:
mfccs = usable_samples['path'].apply(lambda x: get_mfcc(clips + '/' + x))

In [144]:
# Stack matrices into a single tensor
X = np.stack(mfccs)

In [98]:
le = LabelEncoder()
usable_samples['target'] = le.fit_transform(usable_samples['client_id'])
y = usable_samples['target']
ohe = OneHotEncoder(sparse_output = False)
y = ohe.fit_transform(np.array(y).reshape(-1, 1))

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [167]:
# np.save('X_mfccs.npy', X)
# np.save('y_mfccs.npy', y)

In [172]:
usable_samples['target'].value_counts(normalize = True)

target
8    0.168129
3    0.133942
1    0.132179
5    0.110383
7    0.101429
2    0.086190
0    0.085421
6    0.075699
4    0.053405
9    0.053224
Name: proportion, dtype: float64

## Person 2 sample

In [194]:
person_2_sample = usable_samples[usable_samples['target'] == 2]['path'].sample(1).values[0]
Audio(clips + '/' + person_2_sample)

In [192]:
usable_samples[usable_samples['target'] == 2]

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment,target
14857,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_38725143.mp3,"Together, they have four sons who also play fo...",0,0,fourties,female,Scottish English,NaN,en,NaN,2
14858,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_38725144.mp3,"Clitophon is rescued, but the bandits sentence...",0,0,fourties,female,Scottish English,NaN,en,NaN,2
14859,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_38725145.mp3,"Days later, Armstrong left the team.",0,0,fourties,female,Scottish English,NaN,en,NaN,2
14860,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_38725146.mp3,Royal icing is traditionally prepared with fre...,0,0,fourties,female,Scottish English,NaN,en,NaN,2
14861,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_38725147.mp3,He was also nominated for the Pulitzer prize b...,0,1,fourties,female,Scottish English,NaN,en,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
41881,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_39546131.mp3,This is the northernmost reach of the chestnut...,0,0,fourties,female,Scottish English,NaN,en,NaN,2
41882,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_39546132.mp3,Lull would go on to succeed Long in that office.,0,0,fourties,female,Scottish English,NaN,en,NaN,2
41883,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_39546158.mp3,"This land was named Alto Cafezal, or ""High Cof...",0,0,fourties,female,Scottish English,NaN,en,NaN,2
41884,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_39546159.mp3,The divine character of the animal or man is f...,0,0,fourties,female,Scottish English,NaN,en,NaN,2


## MFCC Model 1

In [153]:
def lstm_bidirectional(num_classes):
    
    model = Sequential()

    model.add(Bidirectional(LSTM(units=20, input_shape=(20, 113), return_sequences=True))
    
    model.add(GlobalAveragePooling1D())

    model.add(Flatten())
    
    model.add(Dense(64, activation = 'relu'))

    model.add(Dropout(0.4))

    # Output layer
    model.add(Dense(units=num_classes, activation='softmax'))
    
    model.build(input_shape=(None, 20, 113))

    return model
              
lstm_model_2 = lstm_bidirectional(10)

# Checkpoint
checkpoint_filepath = 'mfcc_checkpoint.model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

# EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,  
    verbose=1,  
    restore_best_weights=True)

# Compile the model
lstm_model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

lstm_model_2.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_10 (Bidirect  (None, 20, 40)            21440     
 ional)                                                          
                                                                 
 global_average_pooling1d_1  (None, 40)                0         
 0 (GlobalAveragePooling1D)                                      
                                                                 
 flatten_10 (Flatten)        (None, 40)                0         
                                                                 
 dense_20 (Dense)            (None, 64)                2624      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_21 (Dense)            (None, 10)              

In [154]:
lstm_model_2.fit(X_train,
               y_train,
               epochs = 30,
                 validation_split = 0.2,
                callbacks = [early_stopping])

Epoch 1/30


2024-03-04 22:55:35.295696: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:35.590605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:35.612476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:35.745788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:35.779724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


443/443 [==============================] - ETA: 0s - loss: 1.6825 - accuracy: 0.3840

2024-03-04 22:55:45.752409: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:45.856674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-04 22:55:45.871495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


443/443 [==============================] - 13s 26ms/step - loss: 1.6825 - accuracy: 0.3840 - val_loss: 1.2587 - val_accuracy: 0.5629
Epoch 2/30
443/443 [==============================] - 11s 25ms/step - loss: 1.1559 - accuracy: 0.5984 - val_loss: 1.0207 - val_accuracy: 0.6652
Epoch 3/30
443/443 [==============================] - 11s 25ms/step - loss: 0.9271 - accuracy: 0.6889 - val_loss: 0.8291 - val_accuracy: 0.7245
Epoch 4/30
443/443 [==============================] - 11s 25ms/step - loss: 0.7875 - accuracy: 0.7444 - val_loss: 0.7505 - val_accuracy: 0.7471
Epoch 5/30
443/443 [==============================] - 11s 25ms/step - loss: 0.6905 - accuracy: 0.7741 - val_loss: 0.6808 - val_accuracy: 0.7751
Epoch 6/30
443/443 [==============================] - 11s 25ms/step - loss: 0.6238 - accuracy: 0.7987 - val_loss: 0.6483 - val_accuracy: 0.7810
Epoch 7/30
443/443 [==============================] - 11s 24ms/step - loss: 0.5683 - accuracy: 0.8152 - val_loss: 0.6040 - val_accuracy: 0.7968
Epo

## MFCC Model 2

In [164]:
def lstm_bidirectional_2(num_classes):
    
    model = Sequential()

    model.add(Bidirectional(LSTM(units=20, input_shape=(20, 113), return_sequences=True,
                                 kernel_regularizer = l2(1e-4),
                                recurrent_dropout = 0.001)))
    
    model.add(GlobalAveragePooling1D())

    model.add(Flatten())
    
    model.add(Dense(10, activation = 'relu', kernel_regularizer = l2(1e-4)))

    model.add(Dropout(0.4))

    # Output layer
    model.add(Dense(units=num_classes, activation='softmax'))
    
    model.build(input_shape=(None, 20, 113))

    return model
              
lstm_model_3 = lstm_bidirectional_2(10)

# Checkpoint
checkpoint_filepath = 'mfcc_checkpoint.model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

# EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,  
    verbose=1,  
    restore_best_weights=True)

# Compile the model
lstm_model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

lstm_model_3.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_16 (Bidirect  (None, 20, 40)            21440     
 ional)                                                          
                                                                 
 global_average_pooling1d_1  (None, 40)                0         
 6 (GlobalAveragePooling1D)                                      
                                                                 
 flatten_16 (Flatten)        (None, 40)                0         
                                                                 
 dense_32 (Dense)            (None, 10)                410       
                                                                 
 dropout_16 (Dropout)        (None, 10)                0         
                                                                 
 dense_33 (Dense)            (None, 10)              

In [165]:
lstm_model_3.fit(X_train,
               y_train,
               epochs = 30,
                 validation_split = 0.2,
                callbacks = [early_stopping])

Epoch 1/30


2024-03-05 09:58:02.781875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


443/443 [==============================] - ETA: 0s - loss: 1.9591 - accuracy: 0.3066

2024-03-05 10:07:26.750514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


443/443 [==============================] - 575s 1s/step - loss: 1.9591 - accuracy: 0.3066 - val_loss: 1.5709 - val_accuracy: 0.4411
Epoch 2/30
 37/443 [=>............................] - ETA: 8:33 - loss: 1.6003 - accuracy: 0.4231

KeyboardInterrupt: 